https://www.youtube.com/watch?v=KYFfMymTspE

In [1]:
from keras.applications.densenet import DenseNet121
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator 
from keras import backend as K

Using TensorFlow backend.
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529

In [2]:
img_rows,img_cols = 224, 224

densenet = DenseNet121(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

for layer in densenet.layers:
    layer.trainable=False

def addTopModeldensenet(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 7

FC_Head = addTopModeldensenet(densenet,num_classes)

model = Model(inputs = densenet.input, outputs = FC_Head)

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
_____________________________________

In [3]:
train_data_dir = 'C:\\Users\\user\\AI_vision\\project_carrot\\CNN\\data\\train_skeleton'
validation_data_dir = 'C:\\Users\\user\\AI_vision\\project_carrot\\CNN\\data\\test_skeleton'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint(
                             'finger_sign_densenet121.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          restore_best_weights=True,
                          patience=10,
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 1200
nb_validation_samples = 300  

epochs = 20
batch_size = 32

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)


Found 4165 images belonging to 7 classes.
Found 1785 images belonging to 7 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
37/37 [==============================] - 297s 8s/step - loss: 1.5219 - acc: 0.3742 - val_loss: 1.7284 - val_acc: 0.3403

Epoch 00001: val_loss improved from inf to 1.72841, saving model to finger_sign_densenet121.h5
Epoch 2/20
37/37 [==============================] - 290s 8s/step - loss: 0.8569 - acc: 0.5971 - val_loss: 2.1173 - val_acc: 0.2569

Epoch 00002: val_loss did not improve from 1.72841
Epoch 3/20
37/37 [==============================] - 282s 8s/step - loss: 0.7608 - acc: 0.6225 - val_loss: 1.6486 - val_acc: 0.2743

Epoch 00003: val_loss improved from 1.72841 to 1.64858, saving model to finger_sign_densenet121.h5
Epoch 4/20
37/37 [==============================] - 267s 7s/step - loss: 0.6162 - acc: 0.6474 - val_loss: 2.5745 - val_acc: 0.1806

Epoch 00004: val_loss did not improve from 1.64858
Epoch 5/20
37/37 [==========================